In [1]:
from typing import List, Literal, Optional, Union, Any, Annotated
from pydantic import BaseModel, Field


class BaseCondition(BaseModel):
    condition_type: str

class BypassCheck(BaseCondition):
    condition_type: Literal["Bypass"] = "Bypass"

class ParameterCheck(BaseCondition):
    condition_type: Literal["Parameter"] = "Parameter"
    category: str
    parameter: str
    comparator: Literal["<", "<=", ">=", ">", "="]
    value: Any

class TimeCheck(BaseCondition):
    condition_type: Literal["Time"] = "Time"
    wait: int

class ButtonPress(BaseCondition):
    condition_type: Literal["Button"] = "Button"
    button_id: str
    
class Lambda(BaseCondition):
    condition_type: Literal["Lambda"] = "Lambda"
    exec_str: str
    

Condition = Annotated[
    Union[BypassCheck, ParameterCheck, TimeCheck, ButtonPress, Lambda, "AndCondition", "OrCondition"],
    Field(discriminator="condition_type")
]

class AndCondition(BaseCondition):
    condition_type: Literal["And"] = "And"
    conditions: List[Condition]

class OrCondition(BaseCondition):
    condition_type: Literal["Or"] = "Or"
    conditions: List[Condition]

Index = int

class EdgeNode(BaseModel):
    local_index: int
    story_name: Optional[str] = None

class Edge(BaseModel):
    start: EdgeNode 
    end: EdgeNode
    condition: Condition = BypassCheck()

class BasePage(BaseModel):
    page_type: Literal["static", "dynamic"]
    parameters: dict[str, Any] = {}

class StaticPage(BasePage):
    page_type: Literal["static"] = "static"

class DynamicPageAbstract(BasePage):
    page_type: Literal["dynamic"] = "dynamic"
    dynamic_type: Literal["model", "none"]

class DynamicPage(DynamicPageAbstract):
    dynamic_type: Literal["none"] = "none"

class ModelPage(DynamicPageAbstract):
    dynamic_type: Literal["model"] = "model"
    model_name: str
    component_type: Literal["train", "predict"]

DynamicPageUnion = Annotated[
    Union[DynamicPage, ModelPage],
    Field(discriminator="dynamic_type")
]

PageUnion = Annotated[
    Union[StaticPage, DynamicPageUnion],
    Field(discriminator="page_type")
]

class StoryNode(BaseModel):
    index: Index
    
class Story(BaseModel):
    name: str
    description: str
    start_page: int = 0
    nodes: list[StoryNode]
    edges: list[Edge]

class Config(BaseModel):
    stories: dict[str, Story]
    pages: dict[Index, PageUnion]

In [2]:
import hashlib

def compute_object_hash(item: Union[PageUnion, Story]) -> int:
    item_json = item.model_dump_json()
    sha256_hash = hashlib.sha256(item_json.encode()).hexdigest()
    return int(sha256_hash[:8], 16)

In [21]:
pages = [StaticPage(parameters={
                        "text": "Test text page"
              }),
         ModelPage(component_type="train",
                   model_name="decision_tree",
              parameters={
                  "whitelist": ["max_depth"]
              }),
         ModelPage(component_type="predict",
                   model_name="decision_tree"),
         StaticPage(parameters={
                        "text": "New story"
              })]

page_dict = {compute_object_hash(p): p for p in pages}

page_dict

{881648388: StaticPage(page_type='static', parameters={'text': 'Test text page'}),
 3595475928: ModelPage(page_type='dynamic', parameters={'whitelist': ['max_depth']}, dynamic_type='model', model_name='decision_tree', component_type='train'),
 413280384: ModelPage(page_type='dynamic', parameters={}, dynamic_type='model', model_name='decision_tree', component_type='predict'),
 3204261850: StaticPage(page_type='static', parameters={'text': 'New story'})}

In [22]:
nodes = [
     StoryNode(index=compute_object_hash(pages[0])),
     StoryNode(index=compute_object_hash(pages[1])),
     StoryNode(index=compute_object_hash(pages[2])),
     StoryNode(index=compute_object_hash(pages[1])),
     StoryNode(index=compute_object_hash(pages[2])),
]


edges = [Edge(start=EdgeNode(local_index=0), 
              end=EdgeNode(local_index=1)),
         Edge(start=EdgeNode(local_index=1), 
              end=EdgeNode(local_index=2), 
              condition=ParameterCheck(parameter="max_depth",
                                       category="trainParams",
                                        comparator="=",
                                        value=3)),
         Edge(start=EdgeNode(local_index=3), 
              end=EdgeNode(local_index=4), 
              condition=ParameterCheck(parameter="max_depth",
                                       category="trainParams",
                                        comparator="=",
                                        value=4)),
         Edge(start=EdgeNode(local_index=2), 
              end=EdgeNode(local_index=3),
              condition=ParameterCheck(parameter="petal length (cm)",
                                       category="predictParams",
                                        comparator="=",
                                        value=2.45)),
         Edge(start=EdgeNode(local_index=4), 
              end=EdgeNode(local_index=0),
              condition=ParameterCheck(parameter="petal length (cm)",
                                       category="predictParams",
                                        comparator="=",
                                        value=2.45)),
         Edge(start=EdgeNode(local_index=4), 
              end=EdgeNode(local_index=0, story_name="decision_tree"))]

story = Story(name="decision_tree_max_depth", 
              description="Decision Tree max depth",
              nodes=nodes,
              edges=edges)

story

Story(name='decision_tree_max_depth', description='Decision Tree max depth', start_page=0, nodes=[StoryNode(index=881648388), StoryNode(index=3595475928), StoryNode(index=413280384), StoryNode(index=3595475928), StoryNode(index=413280384)], edges=[Edge(start=EdgeNode(local_index=0, story_name=None), end=EdgeNode(local_index=1, story_name=None), condition=BypassCheck(condition_type='Bypass')), Edge(start=EdgeNode(local_index=1, story_name=None), end=EdgeNode(local_index=2, story_name=None), condition=ParameterCheck(condition_type='Parameter', category='trainParams', parameter='max_depth', comparator='=', value=3)), Edge(start=EdgeNode(local_index=3, story_name=None), end=EdgeNode(local_index=4, story_name=None), condition=ParameterCheck(condition_type='Parameter', category='trainParams', parameter='max_depth', comparator='=', value=4)), Edge(start=EdgeNode(local_index=2, story_name=None), end=EdgeNode(local_index=3, story_name=None), condition=ParameterCheck(condition_type='Parameter', 

In [23]:
nodes2 = [
     StoryNode(index=compute_object_hash(pages[3])),
]


edges2 = [
     Edge(start=EdgeNode(local_index=0), 
              end=EdgeNode(local_index=0, story_name="decision_tree_max_depth"))
     ]
         

story2 = Story(name="decision_tree", 
              description="Decision Tree",
              nodes=nodes2,
              edges=edges2)

story2

Story(name='decision_tree', description='Decision Tree', start_page=0, nodes=[StoryNode(index=3204261850)], edges=[Edge(start=EdgeNode(local_index=0, story_name=None), end=EdgeNode(local_index=0, story_name='decision_tree_max_depth'), condition=BypassCheck(condition_type='Bypass'))])

In [ ]:
config = Config(stories={story.name: story, story2.name: story2},
                pages=page_dict)

config

Config(stories={'decision_tree_max_depth': Story(name='decision_tree_max_depth', description='Decision Tree max depth', start_page=0, nodes=[StoryNode(index=881648388), StoryNode(index=3595475928), StoryNode(index=413280384), StoryNode(index=3595475928), StoryNode(index=413280384)], edges=[Edge(start=EdgeNode(local_index=0, story_name=None), end=EdgeNode(local_index=1, story_name=None), condition=BypassCheck(condition_type='Bypass')), Edge(start=EdgeNode(local_index=1, story_name=None), end=EdgeNode(local_index=2, story_name=None), condition=ParameterCheck(condition_type='Parameter', category='trainParams', parameter='max_depth', comparator='=', value=3)), Edge(start=EdgeNode(local_index=3, story_name=None), end=EdgeNode(local_index=4, story_name=None), condition=ParameterCheck(condition_type='Parameter', category='trainParams', parameter='max_depth', comparator='=', value=4)), Edge(start=EdgeNode(local_index=2, story_name=None), end=EdgeNode(local_index=3, story_name=None), condition=

In [25]:
with open("frontend/src/assets/config.json", "w", encoding="utf-8") as f:
    f.write(config.model_dump_json(indent=4))